In [1]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import time

In [3]:
df = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding = "latin-1", low_memory=False)

In [4]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [5]:
# rename columns
df.columns = ['target', 'ids', 'date', 'Query', 'user', 'text']

In [6]:
df.head()

,target,ids,date,Query,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [7]:
# move id column to the most left, move target to the most right
cols = df.columns.tolist()
# cols = cols[1] + cols[2] + cols[4] + cols[5] + cols[0]
cols = cols[1:2] + cols[2:3] + cols[4:5] + cols[5:6] + cols[0:1]
df = df[cols]

In [8]:
df.head()

,ids,date,user,text,target
0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...,0
1,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...,0
2,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire,0
3,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all....",0
4,1467811372,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,@Kwesidei not the whole crew,0


In [9]:
# drop nan rows
df = df.dropna()

In [10]:
len(pd.unique(df['target']))

2

In [11]:
len(pd.unique(df['user']))

659775

In [12]:
# groupby users into a new dataframe
group_df = df.groupby('user')

In [13]:
group_df.head()

,ids,date,user,text,target
0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...,0
1,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...,0
2,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire,0
3,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all....",0
4,1467811372,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,@Kwesidei not the whole crew,0
...,...,...,...,...,...
1599993,2193579489,Tue Jun 16 08:39:00 PDT 2009,EvolveTom,"@Cliff_Forster Yeah, that does work better tha...",4
1599994,2193601966,Tue Jun 16 08:40:49 PDT 2009,AmandaMarie1028,Just woke up. Having no school is the best fee...,4
1599995,2193601969,Tue Jun 16 08:40:49 PDT 2009,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,4
1599996,2193601991,Tue Jun 16 08:40:49 PDT 2009,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,4


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [15]:
# train test split
train_x, test_x, train_y, test_y = train_test_split(df['text'], df['target'], test_size=0.2, random_state=5)

In [16]:
# tfidf
tfidf = TfidfVectorizer(min_df=5, max_df=0.8, sublinear_tf=True, use_idf=True)
train_x_tfidf = tfidf.fit_transform(train_x)
test_x_tfidf = tfidf.transform(test_x)

In [17]:
# logistic regression
from sklearn.linear_model import LogisticRegression
def logistic_regression(train_x, train_y):
  classifier = LogisticRegression(max_iter=100000)
  classifier.fit(train_x, train_y)
  return classifier

# svm
from sklearn.svm import SVC
def svm(train_x, train_y):
    # take a subset of train_x and train_y
    train_x = train_x[:20000]
    train_y = train_y[:20000]
    classifier = SVC(max_iter=100000, C=1.0)
    classifier.fit(train_x, train_y)
    return classifier

# knn
from sklearn.neighbors import KNeighborsClassifier
def knn(train_x, train_y):
  # take a subset of train_x and train_y
  train_x = train_x[:100000]
  train_y = train_y[:100000]
  classifier = KNeighborsClassifier()
  classifier.fit(train_x, train_y)
  return classifier

# decision tree
from sklearn import tree
def decision_tree(train_x, train_y):
  train_x = train_x[:10000]
  train_y = train_y[:10000]
  classifier = tree.DecisionTreeClassifier()
  classifier.fit(train_x, train_y)
  return classifier

# gnb
# Not enough memory
# from sklearn.naive_bayes import GaussianNB
# def gnb(train_x, train_y):
#   classifier = GaussianNB()
#   classifier.fit(train_x, train_y)
#   return classifier

In [18]:
# run logistic regression
import time
start_time = time.time()
classifier = logistic_regression(train_x_tfidf, train_y)
log_predictions = classifier.predict(test_x_tfidf)
print(accuracy_score(test_y, log_predictions))
print("--- %s seconds ---" % (time.time() - start_time))

0.801171875
--- 129.72828245162964 seconds ---


In [19]:
# run svm
start_time = time.time()
classifier = svm(train_x_tfidf, train_y)
predictions = classifier.predict(test_x_tfidf[:2000])
print(accuracy_score(test_y[:2000], predictions))
print("--- %s seconds ---" % (time.time() - start_time))

0.773
--- 111.04647922515869 seconds ---


In [20]:
# run knn
start_time = time.time()
classifier = knn(train_x_tfidf, train_y)
predictions = classifier.predict(test_x_tfidf[:10000])
print(accuracy_score(test_y[:10000], predictions))
print("--- %s seconds ---" % (time.time() - start_time))

0.5584
--- 24.126949310302734 seconds ---


In [21]:
# run decision tree
start_time = time.time()
classifier = decision_tree(train_x_tfidf, train_y)
predictions = classifier.predict(test_x_tfidf[:2000])
print(accuracy_score(test_y[:2000], predictions))
print("--- %s seconds ---" % (time.time() - start_time))

0.66
--- 3.337719678878784 seconds ---


In [22]:
# run gnb
# Not enough memory
# start_time = time.time()
# train_x_tfidf_random = train_x_tfidf[:1000]
# train_x_tfidf_random = train_x_tfidf_random.toarray()
# classifier = gnb(train_x_tfidf_random, train_y[:1000])
# test_x_tfidf_gnb = test_x_tfidf[:100]
# predictions = classifier.predict(test_x_tfidf.toarray())
# print(accuracy_score(test_y[:300], predictions))
# print("--- %s seconds ---" % (time.time() - start_time))